In [2]:
import tensorflow as tf

2024-07-18 19:12:16.727281: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle

In [4]:
data=pd.read_csv('Churn_Modelling.csv')

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

In [8]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [10]:
ohe=OneHotEncoder(sparse_output=False)
geo_encoded=ohe.fit_transform(data[['Geography']])
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_df=pd.DataFrame(geo_encoded,columns=ohe.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [13]:
data=pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
#Saving encoders and scaler
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('one_hot_encoder.pkl','wb') as file:
    pickle.dump(ohe,file)

In [15]:
x=data.drop('Exited',axis=1)
y=data['Exited']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [16]:
#Saving Scaler
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN implementation

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),#HL1
    Dense(32,activation='relu'),#HL2
    Dense(1,activation='sigmoid')#output layer
]
)

/Users/arpitsudhir/Downloads/ANN_Project/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [21]:
model.compile(
    optimizer=opt,
    loss="binary_crossentropy",
    metrics=['accuracy']
)

In [22]:
#Setting up TensorBoard
import os
log_dir=os.path.join("/Users/arpitsudhir/Downloads/ANN_Project/logs/",
                     datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [23]:
#Setting up EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [24]:
#Train the model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),
    epochs=50,
    callbacks=[tensorflow_callback,early_stopping_callback]
)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7981 - loss: 0.4529 - val_accuracy: 0.8635 - val_loss: 0.3363
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8515 - loss: 0.3624 - val_accuracy: 0.8495 - val_loss: 0.3582
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8530 - loss: 0.3543 - val_accuracy: 0.8515 - val_loss: 0.3557
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8608 - loss: 0.3435 - val_accuracy: 0.8605 - val_loss: 0.3276
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8623 - loss: 0.3311 - val_accuracy: 0.8675 - val_loss: 0.3309
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8544 - loss: 0.3385 - val_accuracy: 0.8675 - val_loss: 0.3577
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8643 - loss: 0.3342 - val_accuracy: 0.8655 - val_loss: 0.3254
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8669 - loss: 0.3314 - val_accuracy: 0.

In [25]:
model.save('ann_model.h5')

In [26]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/

Reusing TensorBoard on port 6007 (pid 87679), started 2:40:16 ago. (Use '!kill 87679' to kill it.)

In [28]:
#Loading our model
from tensorflow.keras.models import load_model

In [29]:
model=load_model('ann_model.h5')
with open('label_encoder.pkl','rb') as file:
    label_encoder=pickle.load(file)

with open('one_hot_encoder.pkl','rb') as file:
    ohe=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [30]:
input_data={
'CreditScore': 600,

'Geography': 'France',

'Gender': 'Male',

'Age': 40,

'Tenure': 3,

'Balance': 60000,

'NumOfProducts': 2,

'HasCrCard': 1,

'IsActiveMember': 1,

'EstimatedSalary': 50000
}

In [31]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [32]:
geo_array = np.array(input_data["Geography"]).reshape(-1, 1)
geo_encoded=ohe.transform(geo_array)
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe.get_feature_names_out(["Geography"]))
geo_encoded_df


/Users/arpitsudhir/Downloads/ANN_Project/myenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [33]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [34]:

input_df['Gender']=label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [35]:
scaled_input=scaler.transform(input_df)

In [36]:
scaled_input

array([[-0.52306286,  0.91393836,  0.10204461, -0.69055415, -0.25899362,
         0.80499245,  0.65114845,  0.96534994, -0.87091193,  0.989802  ,
        -0.57619557, -0.57061429]])

In [37]:
prediction_prob=model.predict(scaled_input)
prediction_prob

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


array([[0.05844038]], dtype=float32)

In [38]:
if(prediction_prob>0.5):
    print("Customer is likely to churn ")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
